In [1]:
# --- 診断用スクリプト ---
import pandas as pd
import sqlite3
from tqdm.auto import tqdm

# --- 設定（元のスクリプトからコピー） ---
DB_PATH = "data/processed/s2orc_filtered.db"
EVAL_PAPERS_FILE = "data/datapapers/sampled/evaluation_data_papers_50.csv"

# --- 調査開始 ---
print("="*50)
print("--- Starting Investigation: Why 46 instead of 50? ---")
print("="*50)

# 1. 評価対象の全データペーパーDOIをロード
df_eval_papers = pd.read_csv(EVAL_PAPERS_FILE)
all_data_paper_dois = tuple(df_eval_papers['cited_datapaper_doi'].unique())

print(f"Found {len(all_data_paper_dois)} unique data paper DOIs in the input file.\n")

valid_queries_count = 0
skipped_queries = []

# 2. 各データペーパーDOIをループでチェック
with sqlite3.connect(DB_PATH) as conn:
    for i, data_paper_doi in enumerate(tqdm(all_data_paper_dois, desc="Checking each DOI")):
        print(f"\n--- [{i+1}/{len(all_data_paper_dois)}] Checking DOI: {data_paper_doi} ---")
        
        # --- 原因1のチェック ---
        # このデータペーパーを引用し、かつ人間が「正解」とアノテーションした論文を全て取得
        query_gt = "SELECT citing_doi FROM positive_candidates WHERE cited_datapaper_doi = ? AND human_annotation_status = 1"
        gt_rows = conn.execute(query_gt, (data_paper_doi,)).fetchall()
        ground_truth_dois = {row[0] for row in gt_rows}
        
        print(f"Found {len(ground_truth_dois)} annotated positive papers.")

        # 条件: 正解論文が2件以上ないと、クエリと残りの正解に分けられない
        if len(ground_truth_dois) < 2:
            reason = f"Skipped: Not enough positive papers found. (Found {len(ground_truth_dois)}, required >= 2)"
            print(f"🔴 {reason}")
            skipped_queries.append({"doi": data_paper_doi, "reason": reason})
            continue # このDOIの処理を中断し、次に進む

        # --- 原因2のチェック ---
        # 1つをクエリとして取り出す
        query_doi = ground_truth_dois.pop()
        
        # クエリ論文のアブストラクトが存在するかチェック
        query_text_res = conn.execute("SELECT abstract FROM papers WHERE doi = ?", (query_doi,)).fetchone()
        
        if not (query_text_res and query_text_res[0]):
            reason = f"Skipped: The selected query paper ({query_doi}) has no abstract."
            print(f"🔴 {reason}")
            skipped_queries.append({"doi": data_paper_doi, "reason": reason})
            continue

        # --- 原因3のチェック（念のため） ---
        #残りの正解論文がDBに存在するか
        if not ground_truth_dois:
            reason = f"Skipped: After popping one query, no ground truth papers remain."
            print(f"🔴 {reason}")
            skipped_queries.append({"doi": data_paper_doi, "reason": reason})
            continue

        # ここまで到達すれば有効なクエリ
        print(f"✅ Success: This DOI will generate a valid query.")
        print(f"   - Query DOI: {query_doi}")
        print(f"   - Number of ground truths: {len(ground_truth_dois)}")
        valid_queries_count += 1

print("\n" + "="*50)
print("--- Investigation Complete ---")
print("="*50)

print(f"\nTotal valid queries that can be generated: {valid_queries_count}")
print(f"Total skipped data paper DOIs: {len(skipped_queries)}")

if skipped_queries:
    print("\n--- Details of Skipped DOIs ---")
    for item in skipped_queries:
        print(f"- DOI: {item['doi']}")
        print(f"  Reason: {item['reason']}")
else:
    print("\nNo DOIs were skipped.")

--- Starting Investigation: Why 46 instead of 50? ---
Found 50 unique data paper DOIs in the input file.



Checking each DOI:   0%|          | 0/50 [00:00<?, ?it/s]


--- [1/50] Checking DOI: 10.1007/S11558-019-09344-2 ---
Found 45 annotated positive papers.
✅ Success: This DOI will generate a valid query.
   - Query DOI: 10.21203/RS.3.RS-178902/V1
   - Number of ground truths: 44

--- [2/50] Checking DOI: 10.5194/ESSD-13-3907-2021 ---
Found 28 annotated positive papers.
✅ Success: This DOI will generate a valid query.
   - Query DOI: 10.3390/IJERPH20010748
   - Number of ground truths: 27

--- [3/50] Checking DOI: 10.5194/ESSD-14-1917-2022 ---
Found 1 annotated positive papers.
🔴 Skipped: Not enough positive papers found. (Found 1, required >= 2)

--- [4/50] Checking DOI: 10.1002/GDJ3.44 ---
Found 21 annotated positive papers.
✅ Success: This DOI will generate a valid query.
   - Query DOI: 10.3390/ENVIRONSCIPROC2023026088
   - Number of ground truths: 20

--- [5/50] Checking DOI: 10.1016/J.DIB.2017.10.024 ---
Found 13 annotated positive papers.
✅ Success: This DOI will generate a valid query.
   - Query DOI: 10.1016/J.NEUROBIOLAGING.2021.11.004
 